In [1]:
%load_ext autoreload
%autoreload 2

In [60]:
import os
import sys
sys.path.append("..")  # add top folder to path

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import moses
import gentrl

import tinymolecule as tiny

## Initializing the encoder multilayer perceptron (MLP) and decoder MLP to form the variational autoencoder
We note that the latent size for decoder is twice as small because we're drawing a sample from a distribution, whereas for the encoder, we get two values: mean and log variance. The MLPs are relatively small neural networks, by default there are two hidden layers, each with 100 neurons, and output layer (latent space) is 10 neurons. The decoder shares a similar structure but in reverse: the input from latent space is 5 neurons, with two hidden layers of 100 neurons.

In [3]:
enc = tiny.LinearEncoder(latent_size=10)  # mu and sigma
dec = tiny.LinearDecoder(latent_size=5)  # a single sample
vae = tiny.TinyVAE(enc, dec)

In [4]:
train = moses.get_dataset('train')
train_encoded = gentrl.tokenizer.encode(train)[0].float()

In [5]:
train_encoded[0:3]

tensor([[ 1., 23., 23., 23., 25., 22., 13.,  9., 15., 10., 24., 10., 10., 10.,
          3.,  7., 18., 21., 11., 10., 22., 13.,  6., 23., 22., 13.,  9., 15.,
          9., 23., 15.,  7., 18., 21., 11., 10.,  3., 10., 24.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 23., 22., 23., 15., 22., 23., 15., 23., 22., 13.,  9., 15.,
         23., 22.,  9., 10., 24., 10., 10., 10., 22.,  5., 15., 10., 10., 24.,
         15., 18., 24., 10., 10., 18., 10., 24.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 10., 24., 10., 22.,  5., 15., 10., 10., 10., 10., 24.,  6.,
         10., 24., 18., 10., 10., 10., 10., 24., 23., 22., 13.,  9., 15.,  9.,
         23., 23., 22.,  9., 15., 23.,  9.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]])

In [6]:
len(vae(train_encoded[0:3])[0][0])

50

In [52]:
molecloader = tiny.dataset.molecloader(train_encoded)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = nn.BCELoss(reduction='sum')

In [53]:
train_loss = tiny.train.simple_train(vae, molecloader, criterion, optimizer)

epoch 1/5 started at 0.0000 s


RuntimeError: all elements of input should be between 0 and 1

In [43]:
vae(train_encoded[0:3])[0]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       grad_fn=<SigmoidBackward>)

In [24]:
train_encoded[0:3]

tensor([[ 1., 23., 23., 23., 25., 22., 13.,  9., 15., 10., 24., 10., 10., 10.,
          3.,  7., 18., 21., 11., 10., 22., 13.,  6., 23., 22., 13.,  9., 15.,
          9., 23., 15.,  7., 18., 21., 11., 10.,  3., 10., 24.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 23., 22., 23., 15., 22., 23., 15., 23., 22., 13.,  9., 15.,
         23., 22.,  9., 10., 24., 10., 10., 10., 22.,  5., 15., 10., 10., 24.,
         15., 18., 24., 10., 10., 18., 10., 24.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 10., 24., 10., 22.,  5., 15., 10., 10., 10., 10., 24.,  6.,
         10., 24., 18., 10., 10., 10., 10., 24., 23., 22., 13.,  9., 15.,  9.,
         23., 23., 22.,  9., 15., 23.,  9.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]])

In [45]:
a = pd.read_csv("/Users/Munchic/Developer/Capstone/tinymolecule/data/ccr4_ic50_meta.csv")

In [44]:
gentrl.tokenizer.decode(vae(train_encoded[0:3])[0])

['', '', '']

In [58]:
torch.sigmoid(train_encoded[0:3])

tensor([[0.7311, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9526, 0.9991, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 0.9975, 1.0000, 1.0000, 1.0000, 0.9999,
         1.0000, 0.9999, 1.0000, 1.0000, 0.9991, 1.0000, 1.0000, 1.0000, 1.0000,
         0.9526, 1.0000, 1.0000, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808,
         0.8808, 0.8808, 0.8808, 0.8808, 0.8808],
        [0.7311, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9933, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         0.8808, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808, 0.8808,
         0.8808, 0.8808, 0.8808, 0.8808, 0.8808],
        [0.7311, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9933, 1.0000, 1.0000,
         

In [49]:
train_encoded[0:3]

tensor([[ 1., 23., 23., 23., 25., 22., 13.,  9., 15., 10., 24., 10., 10., 10.,
          3.,  7., 18., 21., 11., 10., 22., 13.,  6., 23., 22., 13.,  9., 15.,
          9., 23., 15.,  7., 18., 21., 11., 10.,  3., 10., 24.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 23., 22., 23., 15., 22., 23., 15., 23., 22., 13.,  9., 15.,
         23., 22.,  9., 10., 24., 10., 10., 10., 22.,  5., 15., 10., 10., 24.,
         15., 18., 24., 10., 10., 18., 10., 24.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 10., 24., 10., 22.,  5., 15., 10., 10., 10., 10., 24.,  6.,
         10., 24., 18., 10., 10., 10., 10., 24., 23., 22., 13.,  9., 15.,  9.,
         23., 23., 22.,  9., 15., 23.,  9.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]])

In [66]:
x = F.normalize(train_encoded[0:3])

In [67]:
x

tensor([[0.0100, 0.2304, 0.2304, 0.2304, 0.2504, 0.2204, 0.1302, 0.0901, 0.1502,
         0.1002, 0.2404, 0.1002, 0.1002, 0.1002, 0.0300, 0.0701, 0.1803, 0.2103,
         0.1102, 0.1002, 0.2204, 0.1302, 0.0601, 0.2304, 0.2204, 0.1302, 0.0901,
         0.1502, 0.0901, 0.2304, 0.1502, 0.0701, 0.1803, 0.2103, 0.1102, 0.1002,
         0.0300, 0.1002, 0.2404, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200,
         0.0200, 0.0200, 0.0200, 0.0200, 0.0200],
        [0.0095, 0.2195, 0.2195, 0.2099, 0.2195, 0.1431, 0.2099, 0.2195, 0.1431,
         0.2195, 0.2099, 0.1240, 0.0859, 0.1431, 0.2195, 0.2099, 0.0859, 0.0954,
         0.2290, 0.0954, 0.0954, 0.0954, 0.2099, 0.0477, 0.1431, 0.0954, 0.0954,
         0.2290, 0.1431, 0.1717, 0.2290, 0.0954, 0.0954, 0.1717, 0.0954, 0.2290,
         0.0191, 0.0191, 0.0191, 0.0191, 0.0191, 0.0191, 0.0191, 0.0191, 0.0191,
         0.0191, 0.0191, 0.0191, 0.0191, 0.0191],
        [0.0105, 0.2409, 0.1047, 0.2514, 0.1047, 0.2304, 0.0524, 0.1571, 0.1047,
         